In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
import matplotlib.pyplot as plt
from lstm_cnn_net import lstm_cnn
from lstm_net import LSTM
from cnn_net import MyNet

In [9]:
names = ['AC', 'AD', 'BC', 'BD']
types = ['train', 'val', 'test']
#load data
#mac: /Users/syunsei/Desktop/SII2025/process_data/classifier/
#win: C:\Github_LIU\SII2025\process_data\classifier\
for name in names:
    for type in types:
        globals()[name + '_' + type] = np.load('/Users/syunsei/Desktop/SII2025/process_data/classifier/' + name + '_' + type + '.npy')
        
X_train = np.concatenate((AC_train, AD_train, BC_train, BD_train), axis=0)
X_val = np.concatenate((AC_val, AD_val, BC_val, BD_val), axis=0)
X_test = np.concatenate((AC_test, AD_test, BC_test, BD_test), axis=0)
y_train = np.concatenate((np.zeros(AC_train.shape[0]), np.ones(AD_train.shape[0]), np.ones(BC_train.shape[0]) * 2, np.ones(BD_train.shape[0]) * 3), axis=0)
y_val = np.concatenate((np.zeros(AC_val.shape[0]), np.ones(AD_val.shape[0]), np.ones(BC_val.shape[0]) * 2, np.ones(BD_val.shape[0]) * 3), axis=0)
y_test = np.concatenate((np.zeros(AC_test.shape[0]), np.ones(AD_test.shape[0]), np.ones(BC_test.shape[0]) * 2, np.ones(BD_test.shape[0]) * 3), axis=0)

#归一化
def min_max_normalize(data):
    min_val = np.min(data, axis=0)
    max_val = np.max(data, axis=0)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1  # 防止除零错误
    return (data - min_val) / range_val
#X_train = min_max_normalize(X_train)
#X_val = min_max_normalize(X_val)
#X_test = min_max_normalize(X_test)

#split data into training and testing and validation
'''X_train=np.concatenate ((X_train[:,:,0:3],X_train[:,:,6:9]), axis=2)
X_val=np.concatenate ((X_val[:,:,0:3],X_val[:,:,6:9]), axis=2)
X_test=np.concatenate ((X_test[:,:,0:3],X_test[:,:,6:9]), axis=2)'''
X_train=torch.tensor(X_train,dtype=torch.float32)
X_val=torch.tensor(X_val,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
y_train=torch.tensor(y_train,dtype=torch.long)
y_val=torch.tensor(y_val,dtype=torch.long)
y_test=torch.tensor(y_test,dtype=torch.long)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)


'''#draw X_train
plt.plot(X[1110])
plt.show()

batch_size = 100
train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val))
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)'''

#dataset
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=10)
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, shuffle=True, batch_size=20)
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, shuffle=True, batch_size=20)


torch.Size([715, 301, 12]) torch.Size([46, 301, 12]) torch.Size([47, 301, 12])
torch.Size([715]) torch.Size([46]) torch.Size([47])


In [10]:
#define hyperparameters
sequence_len = 301
input_len = 12
hidden_size = 128
num_layers = 2
num_classes = 4
num_epochs = 100
learning_rate = 0.001

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

Using device: cpu


In [12]:
model = LSTM(input_len, hidden_size, num_layers, num_classes).to(device)
#model = MyNet().to(device)
print(model)

LSTM(
  (lstm): LSTM(12, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=4, bias=True)
  (relu): ReLU()
)


In [13]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
def train(num_epochs, model, train_loader, loss_function):
    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        for batch, (lifts, labels) in enumerate(train_loader):
            for i in range(lifts.size(0)):
                lift = lifts[i]
                '''lifts = lifts.reshape(-1, sequence_len, input_len).to(device)
                labels = labels.long().to(device)'''
                lifts, labels = lift.to(device), label.to(device)
            
            outputs = model(lifts)
            loss = loss_function(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch+1) % 2 == 0:
                loss_list.append(loss.item())
                print(f"Epoch [{epoch+1}; Batch {batch+1}/{total_step}]; Loss: {loss.item():.4f}")
                
loss_list = []
#draw learning curve
def plot_learning_curve(loss_list):
    plt.plot(loss_list, label="loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title("Learning Curve")
    plt.legend()
    plt.show()
    
train(num_epochs, model, train_loader, loss_function)
plot_learning_curve(loss_list)


Epoch [1; Batch 2/72]; Loss: 1.4813
Epoch [1; Batch 4/72]; Loss: 1.3918
Epoch [1; Batch 6/72]; Loss: 1.3675
Epoch [1; Batch 8/72]; Loss: 1.3622
Epoch [1; Batch 10/72]; Loss: 1.3024
Epoch [1; Batch 12/72]; Loss: 1.3601
Epoch [1; Batch 14/72]; Loss: 1.3290
Epoch [1; Batch 16/72]; Loss: 1.2685
Epoch [1; Batch 18/72]; Loss: 1.3055
Epoch [1; Batch 20/72]; Loss: 1.2742
Epoch [1; Batch 22/72]; Loss: 1.4372
Epoch [1; Batch 24/72]; Loss: 1.2883
Epoch [1; Batch 26/72]; Loss: 1.4463
Epoch [1; Batch 28/72]; Loss: 1.4259
Epoch [1; Batch 30/72]; Loss: 1.3259
Epoch [1; Batch 32/72]; Loss: 1.3578
Epoch [1; Batch 34/72]; Loss: 1.3808
Epoch [1; Batch 36/72]; Loss: 1.2707
Epoch [1; Batch 38/72]; Loss: 1.3970
Epoch [1; Batch 40/72]; Loss: 1.2934
Epoch [1; Batch 42/72]; Loss: 1.2409
Epoch [1; Batch 44/72]; Loss: 1.2439
Epoch [1; Batch 46/72]; Loss: 1.2070
Epoch [1; Batch 48/72]; Loss: 1.2634
Epoch [1; Batch 50/72]; Loss: 1.3166
Epoch [1; Batch 52/72]; Loss: 1.1371
Epoch [1; Batch 54/72]; Loss: 1.0146
Epoch

KeyboardInterrupt: 

In [ ]:
def validate(model, val_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for lifts, labels in val_loader:
            lifts = lifts.reshape(-1, sequence_len, input_len).to(device)
            labels = labels.long().to(device)
            
            outputs = model(lifts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f"Validation accuracy: {100 * correct / total}%")

validate(model, val_loader)

Validation accuracy: 78.26086956521739%


In [ ]:
def test(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for lifts, labels in test_loader:
            lifts = lifts.reshape(-1, sequence_len, input_len).to(device)
            labels = labels.long().to(device)
            
            outputs = model(lifts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f"Test accuracy: {100 * correct / total}%")

test(model, test_loader)

Test accuracy: 63.829787234042556%


In [ ]:
#save model
torch.save(model.state_dict(), "model.pth")
